# 结构
- dnn_model(X, Y, layers_dims, learning_rate, num_iterations) return parameters: 模型, 传入属性, 预测值, 网络层结构等, 返回训练好的参数
    - initialize_parameters_deep(layer_dims) return parameters: 初始化网络层参数
    - L_model_forward(X, parameters) return AL, caches: 根据属性和参数, 计算预测值, 可以控制每一层的激活函数是什么, 返回AL为每个样本的预测值, caches为每一层的(A_prev, W, b, Z)
        - linear_activation_forward(A_prev, W, b, activation) return A, cache: 非线性前向传播, 返回值A是本层的输出Z经过激活后的值, cache的值是(A_prev, W, b, Z)
            - linear_forward(A_prev, W, b) return Z, cache: 线性前向传播, A_prev是上一层的输出, W, b是本层的, 通过A_prev, W, b算出本层的输出Z(未激活), 返回的cache值是(A_prev, W, b)
    - compute_cost(AL, Y): 计算成本函数
    - L_model_backward(AL, Y, caches):
        - linear_activation_backward(dA, cache, activation) return dA_prev, dW, db:
            - linear_backward(dZ, cache) return dA_prev, dW, db: 
    - update_parameters(parameters, grads, learning_rate): 调整参数
- predict(X,parameters) return p: 传入训练好的参数来预测

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

# 加载我们自定义的工具函数
from testCases import *
from dnn_utils import *

# 设置一些画图相关的参数
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

In [51]:
# 该函数用于初始化所有层的参数w和b
def initialize_parameters_deep(layer_dims):
    """
    参数:
    layer_dims -- 这个list列表里面，包含了每层的神经元个数。
    例如，layer_dims=[5,4,3]，表示输入层有5个神经元，隐藏层第一层有4个，隐藏层第二层有3个神经元
    
    返回值:
    parameters -- 这个字典里面包含了每层对应的已经初始化了的W和b。
    例如，parameters['W1']装载了第一层的w，parameters['b1']装载了第一层的b
    """
    L = len(layer_dims)  # 一共有多少层
    parameters = {}
    np.random.seed(1)
    
    # 遍历每一层，为每一层的W和b进行初始化
    for i in range(1, L):  # layer_dims[0] 表示输入有多少个神经元, 只需要初始化隐藏层的参数
        # 维度部分可参考笔记: 第i层Wi的维度是(n[i] , n[i-1])
#         parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1]) / np.sqrt(layer_dims[i-1])  # 一行是一个神经元对每个输入的权重, 缩小的程度应该和每个神经元有多少个权重有关, 所以是除以np.sqrt(layer_dims[i-1]), 而不是np.sqrt(layer_dims[i])
        parameters['W'+str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1])  # test
        parameters['b'+str(i)] = np.zeros((layer_dims[i], 1))
        
        # 核对一下W和b的维度是我们预期的维度
        assert(parameters['W'+str(i)].shape == (layer_dims[i], layer_dims[i - 1]))
        assert(parameters['b'+str(i)].shape == (layer_dims[i], 1))
    
    return parameters

In [52]:
# 看看效果
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.72642933 -0.27358579 -0.23620559 -0.47984616  0.38702206]
 [-1.0292794   0.78030354 -0.34042208  0.14267862 -0.11152182]
 [ 0.65387455 -0.92132293 -0.14418936 -0.17175433  0.50703711]
 [-0.49188633 -0.07711224 -0.39259022  0.01887856  0.26064289]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.55030959  0.57236185  0.45079536  0.25124717]
 [ 0.45042797 -0.34186393 -0.06144511 -0.46788472]
 [-0.13394404  0.26517773 -0.34583038 -0.19837676]]
b2 = [[0.]
 [0.]
 [0.]]


In [53]:
# 前向传播, Z^[i] = W^[i]A^[i-1]+b^[i]
def linear_forward(A_prev, W, b):   
    Z = np.dot(W, A_prev) + b
    
    assert(Z.shape == (W.shape[0], A_prev.shape[1]))
    cache = (A_prev, W, b) # 将这些变量保存起来，因为后面进行反向传播时会用到它们
    
    return Z, cache

In [54]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [57]:
# 实现公式 A[i]=g(Z[i])，g代表激活函数，使上面的线性前向传播就变成非线性前向传播。激活函数sigmoid和relu定义在dnn_utils.py中
def linear_activation_forward(A_prev, W, b, activation):
    """
    Arguments:
    A_prev -- 上一层得到的A，输入到本层来计算Z和本层的A。第一层时A_prev就是特征输入X
    W -- 本层相关的W
    b -- 本层相关的b
    activation -- 两个字符串，"sigmoid"或"relu"，指示该层应该使用哪种激活函数
    """
    Z, linear_cache = linear_forward(A_prev, W, b)
    if activation == "sigmoid":
        A = sigmoid(Z)
    elif activation == "relu":
        A = relu(Z)
    else:
        raise ValueError
        
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
#     cache = (linear_cache, Z) # 缓存一些变量，后面的反向传播会用到它们
        
    return A

In [63]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [68]:
# 一个完整的前向传播过程。前向传播一共有L层，前面L-1层用激活函数relu，最后一层用sigmoid
def L_model_forward(X, parameters):
    """
    参数:
    X -- 输入的特征数据
    parameters -- 这个list列表里面包含了每一层的参数w和b
    """
    
    L = len(parameters)//2  # 为什么是一半呢？因为列表是这样的[w1,b1,w2,b2...wl,bl],里面的w1和b1代表了一层
    
    A_prev = X
    for i in range(1, L):  # 先进行L-1次前向传播, 最后一次在下面, i从1开始
        A = linear_activation_forward(A_prev, 
                                           parameters['W'+str(i)],
                                           parameters['b'+str(i)],
                                          "relu")
        A_prev = A
        
    # 进行最后一层的前向传播，这一层的激活函数是sigmoid。得出的AL就是y'预测值
    AL = linear_activation_forward(A_prev, 
                                   parameters['W'+str(L)], 
                                   parameters['b'+str(L)], 
                                  "sigmoid")
    
    assert(AL.shape == (1, X.shape[1]))
    
    return AL

In [69]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]


In [70]:
# 上面已经完成了前向传播了。下面这个函数用于计算成本（单个样本时是损失，多个样本时是成本）
# 通过每次训练的成本我们就可以知道当前神经网络学习的程度好坏。
def compute_cost(AL, Y):
    """
    AL -- 预测值
    Y -- 真实值
    """
    m = Y.shape[1]
    cost = -(1/m) * np.sum(np.multiply(Y, np.log(AL) + np.multiply(1-Y, np.log(1-AL))))
    cost = np.squeeze(cost)  # 确保cost是一个数值而不是一个数组的形式
    
    assert(cost.shape == ())

    return cost

In [71]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


In [ ]:
# 反向传播
def linear_backward(dZ, cache):
    """
    参数:
    dZ -- 后面一层的dZ
    cache -- 前向传播时我们保存下来的关于本层的一些变量
    """